# モデル作成のポイント
- 前月からの売り上げ予測として考える(2015年10月から2015年11月の売り上げをイメージ)
- 月ごとの売り上げdfを作成することを考える
- [lag operator](https://medium.com/@NatalieOlivo/use-pandas-to-lag-your-timeseries-data-in-order-to-examine-causal-relationships-f8186451b3a9)を使って
lagの値を取得してNaNをゼロで埋め、値を0-20の範囲にクリップする。
- 必ずターゲットの値を0-20の値に収めること
- gradient boost treeモデルに突っ込む
- アイテム/ショップペアラグとは別に、あなたは合計ショップまたは合計アイテム売上のラグ値を加えることを試みることができます（それは本質的に平均エンコーディングです）。 

 # 目標
 
 - 2015年11月に各ショップで各アイテムがいくつ売れるかを予測する。
 - 11月の日毎に幾つ売れるか予測してから、それを合算した結果をまとめるのもあり。
 
 ---
 
 目的変数:item_cent_day
 
 説明変数:date, date_block_num, shop_id, item_id, item_price

# メモ

- 説明変数にするために日付を曜日に変換、weekdayかholidayかにするのはあり。
- 特徴を何にするか→曜日、平日or休日、date_block_num、item_id、item_category_id
- 時系列予測みたいなのはかなりきつい(お店x品物分のモデル構築は不可能だし(データ少ない)、汎用的なのは無理)
- date_block_numはテストだと新しくなるけど、大丈夫なのか
- データは日毎ではあるけど、モデルの予測は月ごとである必要がある→こういう場合なるべく情報を持つにはどうすればいいのか

# EDA

*やりたいこと*
- 可視化(各ショップのアイテムがどれだけ売れるかを棒グラフで)

 ## 必要なライブラリの読み込み

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

 ## データの読み込み

In [3]:
path = '/Users/soki/Documents/kaggle/coursera/data/'
# [item_category_name, item_category_id]
item_categories = pd.read_csv(path + 'item_categories.csv')
# [item_name, item_id, item_category_id]
items = pd.read_csv(path + 'items.csv')

# [date, date_block_num, shop_id, item_id, item_price, item_cnt_day]
sales_train = pd.read_csv(path + 'sales_train.csv.gz')
sales_train['date'] = pd.to_datetime(sales_train['date'], format='%d.%m.%Y')
# [shop_name, shop_id]
shops = pd.read_csv(path + 'shops.csv')
# [ID, shop_id, item_id]
test = pd.read_csv(path + 'test.csv.gz')

In [4]:
sales_train.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
5,2013-01-10,0,25,2564,349.00,1.0
6,2013-01-02,0,25,2565,549.00,1.0
7,2013-01-04,0,25,2572,239.00,1.0
8,2013-01-11,0,25,2572,299.00,1.0
9,2013-01-03,0,25,2573,299.00,3.0


In [5]:
test.head(10)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
5,5,5,5039
6,6,5,5041
7,7,5,5046
8,8,5,5319
9,9,5,5003


In [6]:
print('train min/max date: %s/%s'%(min(sales_train['date']), 
                                   max(sales_train['date'])))

train min/max date: 2013-01-01 00:00:00/2015-10-31 00:00:00


## 欠損値の確認

In [7]:
print(item_categories.isnull().any())
print(items.isnull().any())
print(sales_train.isnull().any())
print(shops.isnull().any())
print(test.isnull().any())

item_category_name    False
item_category_id      False
dtype: bool
item_name           False
item_id             False
item_category_id    False
dtype: bool
date              False
date_block_num    False
shop_id           False
item_id           False
item_price        False
item_cnt_day      False
dtype: bool
shop_name    False
shop_id      False
dtype: bool
ID         False
shop_id    False
item_id    False
dtype: bool


 ## データの可視化(店舗ごと) 

In [8]:
# 月ごとに、店舗ごとに、商品ごとにデータを分類
sales_train['revenue'] = sales_train['item_price']*sales_train['item_cnt_day']
new_df = sales_train.drop(['item_price'], axis=1)
new_df.head(10)
grouped = new_df.groupby(['date_block_num', 'shop_id', 'item_id'])
sorted_df = grouped.sum().reset_index()
df0 = sorted_df[sorted_df['date_block_num'] == 0]
df1 = sorted_df[sorted_df['date_block_num'] == 1]

In [9]:
df_dict = {}
df_dict[0] = sorted_df[sorted_df['date_block_num'] == 0]
df_dict[0][df_dict[0]['shop_id'] == 0].head(10)

,date_block_num,shop_id,item_id,item_cnt_day,revenue
0,0,0,32,6.0,1326.0
1,0,0,33,3.0,1041.0
2,0,0,35,1.0,247.0
3,0,0,43,1.0,221.0
4,0,0,51,2.0,257.0
5,0,0,61,1.0,195.0
6,0,0,75,1.0,76.0
7,0,0,88,1.0,76.0
8,0,0,95,1.0,193.0
9,0,0,96,1.0,70.0


In [30]:
item_id_and_category = items[['item_id', 'item_category_id']]
def convert_df(df):
    new_df = pd.DataFrame(item_id_and_category)
    new_df['item_cnt_day'] = 0
    tmp_df = df[['item_id', 'item_cnt_day']]
    for index, row in tmp_df.iterrows():
        new_df.loc[new_df.item_id == row[0], 'item_cnt_day'] = row[1]
    return new_df

In [32]:
df_c = convert_df(df_dict[0][df_dict[0]['shop_id'] == 0])
df_c[df_c.item_id == 32]

,item_id,item_category_id,item_cnt_day
32,32,40,6.0


In [34]:
print(min(shops['shop_id']), max(shops['shop_id']))

0 59


In [35]:
df_dict = {}
for i in range(34):
    for j in range(60):
        tmp_df = sorted_df[sorted_df['date_block_num'] == i]
        tmp_df = tmp_df[tmp_df['shop_id'] == j]
        df_c = convert_df(tmp_df)
        df_dict[(i, j)] = df_c

In [43]:
l = list(df_dict.values())

df_c_concat = pd.concat(l, axis=0)
df_c_concat.to_csv('df_concat.csv')    

# モデル構築

- ランダムフォレストでいけないかな...


In [59]:
# 入力pandas.series、出力曜日(sun:0, sat:6)

# 日付の部分から曜日と平日か休日かのデータに置き換える
df = sales_train.copy()
df['day_of_week'] = df.index.weekday
new_df = pd.merge(df, items)
train_X = new_df.drop(['item_name', 'item_cnt_day'], axis=1)
train_y = new_df['item_cnt_day']

In [61]:
rfr = RandomForestRegressor()
rfr.fit(train_X, train_y)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([1. , 1.2, 1.1, ..., 1. , 1. , 1. ])